# Pipeline 1 - Embedding

### Step 1. Loading

In this step, we load data from various sources. Make them ready to ingest.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DOCUMENT = os.getenv("DOCUMENT")

### Step 2. Parsing

##### Type 1. text document

In [5]:
from langchain.document_loaders import TextLoader
txt_path = DOCUMENT+"rag.txt"
txt_loader = TextLoader(txt_path)
text_documents = txt_loader.load()
#text_documents

##### Type 2. PDF document

We use PyMuPDFLoader in this experiment

In [6]:
from langchain.document_loaders import PyMuPDFLoader
pdf_path = DOCUMENT+ "2005.11401v4.pdf"
pdf_loader = PyMuPDFLoader(pdf_path)
pdf_documents = pdf_loader.load()

### Step 3. Chunking

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(text_documents)
#documents[:3]

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
pdf_chunks = text_splitter.split_documents(pdf_documents)

In [9]:
chunks = text_chunks + pdf_chunks

### Step 4. Vectorizing

Option 1: Using openAI embedding API

In [10]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

In [11]:
embeddings = OpenAIEmbeddings()
vectorstore = DocArrayInMemorySearch.from_documents(chunks, embeddings)

### Step 5. Storing

Trying to persist the vectordb with Chroma

In [12]:
from langchain.vectorstores import Chroma
persist_directory = os.getenv("STORAGE")
vectordb = Chroma.from_documents(documents=pdf_chunks,  embedding=embeddings, persist_directory=persist_directory)
vectordb.persist()

/Users/derektran1/Library/CloudStorage/OneDrive-Personal/1 - Technology/Workspace/rag/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


# Pipline 2. Retrieving

### Step 1. Query

In [13]:
#user_query = "What is retrieval augmented generation"
user_query = "Describe the RAG-Sequence Model?"

### Step 2. Search

Need to load from store if there is. Here the on memory vectorstore is used. 
There is opportunity to improve efficiency of search when the knowledgebase gets larger and more complicated (type of sources)

In [10]:
#retriever = vectorstore.as_retriever()

#Load vectordb from persisted store
from langchain.vectorstores import Chroma
persist_directory = os.getenv("STORAGE")
embeddings = OpenAIEmbeddings()
newvectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)
retriever = newvectordb.as_retriever()

### Step 3. Augmented Prompt

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
setup = RunnableParallel(context=retriever, question=RunnablePassthrough())

### Step 4. Response Generating

Option 1: Using on-cloud OpenAI

In [13]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()

In [14]:
chain = setup | prompt | model | parser

In [18]:
response = chain.invoke(user_query)
response

'The RAG-Sequence Model uses the same retrieved document to generate the complete sequence. It treats the retrieved document as a single latent variable that is marginalized to get the seq2seq probability p(y|x) via a top-K approximation.'

In [20]:
while True:
        user_input = input("Enter a query: ")
        if user_input == "exit":
            break

        try:
            response = chain.invoke(user_input)
            print(response)
        except Exception as err:
            print('Exception occurred. Please try again', str(err))

AWS supports RAG through services such as Amazon Bedrock and Amazon Kendra. Amazon Bedrock offers high-performing foundation models and capabilities to build generative AI applications, while Amazon Kendra provides an optimized Retrieve API for enterprise search powered by machine learning. With knowledge bases for Amazon Bedrock and the Retrieve API from Amazon Kendra, users can connect their data sources for RAG in just a few clicks, enabling vector conversions, retrievals, and improved output generation automatically.
